In [1]:
# -*- coding: utf-8 -*-
from datetime import datetime
import regex as re
from abc import ABCMeta, abstractmethod
from collections import OrderedDict

import jieba
import jieba.posseg as pseg
import requests
from bs4 import BeautifulSoup

In [2]:
# encoding=UTF-8
# !flask/bin/python

from cassandra.cluster import Cluster
from cassandra.policies import DCAwareRoundRobinPolicy
from cassandra.auth import PlainTextAuthProvider
from cassandra.query import BatchStatement
import pandas as pd


class CassandraType(object):
    PRODUCTION = 0
    TEST = 1
    TEST_DOCKER = 2


class CassandraDAO(object):

    # you have to install following items :
    # a. python-Cassandra driver
    # b. pyspark cassandra connector

    def __init__(self, type):
        #         print('runing father.__init__')
        if type == CassandraType.PRODUCTION:
            self.contact_points = ['192.168.95.127', '192.168.95.122']
            self.contact_points_str = "192.168.95.127,192.168.95.122"
        elif type == CassandraType.TEST:
            self.contact_points = ['192.168.0.41', '192.168.0.42']
            self.contact_points_str = "192.168.0.41,192.168.0.42"
        else:
            self.contact_points = ['192.168.0.121', '192.168.0.122', '192.168.0.52']
            self.contact_points_str = "192.168.0.121,192.168.0.122,192.168.0.52"

        self.formatString = "org.apache.spark.sql.cassandra"
        self.username = "username"
        self.password = "password"
        self.cluster = None
        self.session = None
        self.createSession()

    def __del__(self):
        self.cluster.shutdown()

    def pandas_factory(self, colnames, rows):
        return pd.DataFrame(rows, columns=colnames)

    def createSession(self):
        print "contact_points = " + self.contact_points_str
        self.cluster = Cluster(
            contact_points=self.contact_points,  # random select a node
            #             load_balancing_policy = DCAwareRoundRobinPolicy(local_dc='datacenter1'),
            #         auth_provider = PlainTextAuthProvider(username='cassandra', password='cassandra')
        )
        self.session = self.cluster.connect()
        self.session.row_factory = self.pandas_factory
        self.session.default_fetch_size = 10000000  # needed for large queries, otherwise driver will do pagination. Default is 50000.

    def getSession(self):
        return self.session

    def execCQL(self, keyspace, cql):
        """
        execute CQL
        """
        self.session.set_keyspace(keyspace)
        self.session.execute_async(cql)

    def execCQLSelect(self, keyspace, cql):
        """
        execute CQL, select only
        """

        self.session.set_keyspace(keyspace)

        #       cassandra ResultSet
        async_results = self.session.execute_async(cql)
        return async_results

    def execCQLCallBackAnysc(self, keyspace, cql, handle_success, handle_error):
        """
        execute CQL, if success => handle_success function, else handle_error
        """
        self.session.set_keyspace(keyspace)
        async_results = self.session.execute_async(cql)
        async_results.add_callbacks(handle_success, handle_error)

    def execCQLSelectToPandasDF(self, keyspace, cql):
        """
        execute CQL, select only, return Pandas DataFrame
        """

        self.session.set_keyspace(keyspace)

        #       cassandra ResultSet
        async_results = self.session.execute_async(cql)
        #         async_results = self.session.execute_async(cql)
        #       to Pandas DataFrame
        return async_results.result()._current_rows


    def execCQLSelectToRDD(self, sqlContext, keyspace, cql):
        """
        execute CQL, select only, return Spark RDD
        """

        return self.execCQLSelectToDF(sqlContext, keyspace, cql).rdd.map(tuple)  # dataFrame to RDD

    @property
    def contactPoints(self):
        return self.contact_points

    @contactPoints.setter
    def contactPoints(self, contact_points):
        self.contact_points = contact_points

    @contactPoints.deleter
    def contactPoints(self):
        del self.contact_points


In [3]:
CASSANDRA_ENV = CassandraType.PRODUCTION
dao = CassandraDAO(CASSANDRA_ENV)

contact_points = 192.168.95.127,192.168.95.122


In [4]:
import pandas as pd
import jieba
import jieba.posseg as pseg

class JiebaSegmentor:

    def __init__(self, dict_path, userdict=[], stopwords = False, stopwords_path=None):
        self.dict_path = dict_path
        self.userdict = userdict
        self.dictionary_init()
        self.stopwords_path = stopwords_path
        self.stopwords = stopwords
        self.stopwords_set = set()
        self.stopwords_init()

    def dictionary_init(self):
        jieba.set_dictionary(self.dict_path)
        for path in self.userdict:
            print path
            jieba.load_userdict(path)
    
    def stopwords_init(self):
        if self.stopwords_path:
            with open(self.stopwords_path ,'r') as stopwords:
                for stopword in stopwords:
                    self.stopwords_set.add(stopword.strip('\n').decode('utf-8'))

    def taiwan_country(self):
        return [u'臺北', u'台北', u'基隆', u'臺中', u'台中', u'臺南', u'台南', u'高雄',
                u'宜蘭', u'桃園', u'新竹', u'苗栗', u'彰化', u'南投', u'嘉義', u'雲林',
                u'屏東', u'臺東', u'台東', u'花蓮', u'澎湖']

    def wordToNumber(self, input_text):

        target = u''
        for s in input_text:

            if (s == u'零') or (s == '0'):
                to_word = u'0'
            elif (s == u'一') or (s == u'壹') or (s == '1'):
                to_word = u'1'
            elif (s == u'二') or (s == u'兩') or (s == u'貳') or (s == '2'):
                to_word = u'2'
            elif (s == u'三') or (s == u'參') or (s == '3'):
                to_word = u'3'
            elif (s == u'四') or (s == u'肆') or (s == '4'):
                to_word = u'4'
            elif (s == u'五') or (s == u'伍') or (s == '5'):
                to_word = u'5'
            elif (s == u'六') or (s == u'陸') or (s == '6'):
                to_word = u'6'
            elif (s == u'七') or (s == u'柒') or (s == '7'):
                to_word = u'7'
            elif (s == u'八') or (s == u'捌') or (s == '8'):
                to_word = u'8'
            elif (s == u'九') or (s == u'玖') or (s == '9'):
                to_word = u'9'
            else:
                to_word = s

        target = target + to_word
        return target

    def input_text_preprocessing(self, input_text):

        if type(input_text) is not unicode:
            input_text = input_text.decode('utf-8')

#         input_text = self.wordToNumber(input_text)
        return input_text

    def get_names(self, input_text):

        input_text = self.input_text_preprocessing(input_text)
        names = []
        words = pseg.cut(input_text)
        print words
        for w, f in words:
            if f.lower() == 'nr':
                names.append(w)
        for name in names:
            print name.encode('utf-8')
        return names

    def lcut(self, input_text):

        input_text = self.input_text_preprocessing(input_text)
        cut_raw = jieba.lcut(input_text)
        key = []

        for k in cut_raw:
            if self.stopwords:
                if k in self.stopwords_set:
                    continue
            
            key.append(k)
        df = pd.DataFrame({"word": key})
        return df

    def pseg_lcut(self, input_text):

        input_text = self.input_text_preprocessing(input_text)
        cut_raw = pseg.lcut(input_text)
        key = []
        value = []

        for k, v in cut_raw:
            tag = v
            if self.stopwords:
                if k in self.stopwords_set:
                    continue
            
            if k in self.taiwan_country():
                tag = u'ns'
            if len(k) > 1 and tag == u'x':
                tag = u'n'
            key.append(k)
            value.append(tag)
        df = pd.DataFrame({"word": key, "tag": value})
        return df

In [45]:
import jieba
import jieba.posseg as pseg
jieba_path = "/nfs/aq_test/jieba/"
jieba_dict_path1 = jieba_path + "dict_taiwan.txt"
jieba_dict_path2 = jieba_path +"userdict.txt"
jieba_dict_path3 = jieba_path +"dict.txt.big"
jieba_dict_path4 = jieba_path +"dict.txt.small"
jieba_stopwords_path = jieba_path +"stopwords.txt"
segmentor = JiebaSegmentor(jieba_dict_path1, 
                           [jieba_dict_path2,jieba_dict_path3,jieba_dict_path4], 
                           stopwords = False, 
                           stopwords_path=jieba_stopwords_path)

Building prefix dict from /nfs/aq_test/jieba/dict_taiwan.txt ...
DEBUG:jieba:Building prefix dict from /nfs/aq_test/jieba/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.u34e30d1c15f49735c28e2158fa64cd7a.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.u34e30d1c15f49735c28e2158fa64cd7a.cache
Loading model cost 0.578 seconds.
DEBUG:jieba:Loading model cost 0.578 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


/nfs/aq_test/jieba/userdict.txt
/nfs/aq_test/jieba/dict.txt.big
/nfs/aq_test/jieba/dict.txt.small


In [7]:
# jieba & 詞性測試
test_cut =pseg.lcut('西瓜好大一顆')
text_subject_dict={}
for w,n in test_cut:
    print w + ' (' + n + ')'
    text_subject_dict.update({w:n})
print '---subject_dict---'
for k, v in text_subject_dict.items():
    print k + ' (' + v + ')'

西瓜 (ns)
好 (a)
大 (a)
一顆 (m)
---subject_dict---
一顆 (m)
西瓜 (ns)
好 (a)
大 (a)


In [8]:
test_cut_raw_0 = pseg.lcut('陳水扁')
for w,n in test_cut_raw_0:
    print w + ' (' + n + ')'

陳水扁 (nr)


In [216]:
unit_sentence = [u'什麼',u'哪個',u'哪些',u'那個',u'那些',u'哪兒',u'那兒']#,u'哪',u'那']
unit_sentence_rule = ''
for k,v in enumerate(unit_sentence):
    if k == len(unit_sentence) - 1:
        unit_sentence_rule = unit_sentence_rule + v
    else:
        unit_sentence_rule = unit_sentence_rule + v + '|'
print '(' + unit_sentence_rule + ')'

(什麼|哪個|哪些|那個|那些|哪兒|那兒)


In [9]:
class CheckQuestion:
    __metaclass__ = ABCMeta

    def __init__(self):
        pass

    class Label(object):
        WHY = 'why'
        WHEN = 'when'
        WHERE = 'where'
        WHO = 'who'
        WHAT = 'what'
        HOW = 'how'
        HOW_MANY = 'how_many'
        STATUS = 'status'
        FEEL = 'feel'
        OTHER = 'other'

    def get_v_subject(self):
        return ['v', 'vd', 'vg', 'vi', 'vn', 'vq', 'vt']

    def get_adj_subject(self):
        return ['a', 'ad', 'ag', 'an']

    def get_people_subject(self):
        return ['nr','nrfg','nrt']

    def get_location_subject(self):
        return ['ns']
    
    def get_n_subject(self):
        return ['n','ng','nt','nz']
 
    def get_eng_subject(self):
        return ['eng']
    
    def __get_unit_word(self):
        return [u'有沒有',u'什麼', u'哪一個', u'那一個', u'哪個', u'哪些', u'那個', u'那些', u'怎麼', u'哪兒', u'那兒', u'怎', u'哪', u'那']

    def get_adj_word_rule(self):
        return u'(鬼|怪|神祕|神奇)?(的)?'

    def get_unit_word_post_rule(self):

        unit_word = self.__get_unit_word()
        unit_word_rule = ''

        for k, v in enumerate(unit_word):
            unit_word_rule = unit_word_rule + v
            if k < len(unit_word) - 1:
                unit_word_rule = unit_word_rule + '|'

        unit_word_rule = '(' + unit_word_rule + ')' + u'(是)?'

        return unit_word_rule

    def get_unit_word_pre_rule(self):

        unit_word = self.__get_unit_word()
        unit_word_rule = ''

        for k, v in enumerate(unit_word):
            unit_word_rule = unit_word_rule + v
            if k < len(unit_word) - 1:
                unit_word_rule = unit_word_rule + '|'

        unit_word_rule = u'(是)?' + '(' + unit_word_rule + ')'

        return unit_word_rule

    def __repr__(self):
        return "{class_name}".format(class_name=self.__class__.__name__)

    @abstractmethod
    def check(self):
        pass


# 詢問原因

In [10]:
class CheckWhy(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhy, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問原因:
        label: why
        句型：
        1.(為什麼|爲何|爲啥)
        2.通用疑問句+通用形容句+(原因|理由)
        3.(原因|理由) + 通用疑問句+通用形容句
        """
        self.label = ''
        self.target = ''
        
        #1
        rule = u'(為什麼|爲何|爲啥)'
        pattern = re.compile(rule)
        #         print rule
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHY
            return True

        #2
        rule = self.get_unit_word_post_rule() + self.get_adj_word_rule() + u'(原因|理由)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHY
            return True

        #3
        rule = u'(原因|理由)' + self.get_unit_word_pre_rule() + self.get_adj_word_rule()
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHY
            return True

    # cc = CheckWhy(u'小狗會叫的原因是什麼')
    # if cc.check():
    #     print cc.label

# 詢問時間

In [11]:
class CheckWhen(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhen, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問時間:
        label: when
        """
        self.label = ''
        self.target = ''

        rule = u'(什麼時間|什麼時候|什麼年份|什麼月份|什麼年|什麼月|什麼日|什麼天|'+\
                u'哪時|那時|何時|哪一年|哪一月|哪一天|哪年|哪月|哪日|哪天|那一年|那一月|那一天|那年|那月|那日|那天)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            self.label = self.Label.WHEN
            return True

#         rule = self.get_unit_word_post_rule() + \
#                self.get_adj_word_rule() + \
#                u'(時間|時候|年份|月份|一年|一月|一天|年|月|日|天)'
#         #         print rule
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)
#         # print match
#         if match is not None:
#             self.label = self.Label.WHEN
#             return True

#         rule = u'(時間|時候|年份|月份|一年|一月|一天|年|月|日|天)' + \
#                self.get_unit_word_pre_rule() + \
#                self.get_adj_word_rule()
#         #         print rule
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)
#         if match is not None:
#             self.label = self.Label.WHEN
#             return True

    # # cc = CheckWhen(u'什麼年份發生法國大革命')
    # cc = CheckWhen(u'法國大革命發生年份是什麼')
    # if cc.check():
    #     print cc.label

# 詢問地點

In [12]:
class CheckWhere(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhere, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問地點:
        label: where
        句型：
        1.問單一地點
        2.(在何處|在何地|何地|何處|在哪裡|哪裡|在哪兒|哪兒|在哪|在那裡|那裡|在那兒|那兒|在那)
        3.通用疑問句+通用形容句+(地點|地方|國家|省分|城市|城鎮|{詞性為ns}))
        """
        self.label = ''
        self.target = ''
        
        #1
        cut_df = self.segmentor.pseg_lcut(input_text)
        if len(cut_df) == 1 and cut_df['tag'][0] in self.get_location_subject():
            self.label = self.Label.WHERE
            self.target = cut_df['word'][0]
            return True
        
        #2
        rule = u'(在何處|在何地|何地|何處|在哪裡|哪裡|在哪兒|哪兒|在哪|在那裡|那裡|在那兒|那兒|在那)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        if match is not None:
            for index, row in cut_df.iterrows():
                w = row['word']
                n = row['tag']
                if n.lower() in self.get_location_subject(): #or n.lower() in self.get_eng_subject():
                    self.label = self.Label.WHERE
                    self.target = w
                    return True

            self.label = self.Label.WHERE
            return True
        
        #3
        rule = self.get_unit_word_post_rule() + \
               self.get_adj_word_rule() + \
               u'(地點|地方|國家|省分|城市|城鎮)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        # print match
        if match is not None:
            self.label = self.Label.WHERE
            return True

#         rule = self.get_unit_word_post_rule() + u'(.*)'
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)

#         if match is not None:
# #             print match.group(1)
# #             print match.group(2)
# #             print match.group(3)

#             content = None
#             if match.group(2):
#                 content = match.group(2)

#             if match.group(3):
#                 content = match.group(3)

#             if content is not None:
#                 cut_df = self.segmentor.pseg_lcut(content)
#                 for index, row in cut_df.iterrows():
#                     w = row['word']
#                     n = row['tag']
#                     if n.lower() in self.get_location_subject():
#                         self.label = self.Label.WHERE
#                         self.target = w
#                         return True
#                     break

#         rule = u'(.*)' + self.get_unit_word_pre_rule()
#         #         print rule
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)

#         if match is not None:
#             #             print match.group(1)

#             content = None
#             if match.group(1):
#                 content = match.group(1)

#             if content is not None:
#                 cut = self.segmentor.pseg_lcut(content)
#                 for w, n in cut:
#                     #                     print w + ' (' + n + ')'

#                     if n.lower() in self.get_location_subject():
#                         self.label = self.Label.WHERE
#                         return True
#                     break
    
# cc = CheckWhere(segmentor)
# if cc.check(u'[問卦] 有沒有subway塑膠手套ㄉ8卦?'):
#     print cc.label+ ' ; ' + cc.target

# 詢問人

In [35]:
class CheckWho(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWho, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問人:
        label: who
        句型：
        1.問單一人名
        2.(誰|哪位|那位)
        3.通用疑問句+通用形容句+(人)
        4.通用疑問句+通用形容句+{詞性為nr}
        5.{詞性為nr}+通用疑問句+通用形容句
        """
        self.label = ''
        self.target = ''
        
        #1
        cut_df = self.segmentor.pseg_lcut(input_text)
        print(cut_df)
        if len(cut_df) == 1 and cut_df['tag'][0] in self.get_people_subject():
            self.label = self.Label.WHO
            self.target = cut_df['word'][0]
            return True
        
        #2
        rule = u'(是誰|哪位|那位|什麼人|誰是)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        # print match
        if match is not None:
            for index, row in cut_df.iterrows():
                w = row['word']
                n = row['tag']
#                 print w
#                 print n
                if n.lower() in self.get_people_subject(): #or n.lower() in self.get_eng_subject():
                    self.label = self.Label.WHO
                    self.target = w
                    return True

            self.label = self.Label.WHO
            return True
        
#         #3
#         rule = self.get_unit_word_post_rule() + \
#                self.get_adj_word_rule() + \
#                u'(人)'
#         #         print rule
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)
#         # print match
#         if match is not None:
#             self.label = self.Label.WHO
#             return True
        
#         #4
#         rule = self.get_unit_word_post_rule() + u'(.*)'
#         print rule
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)

#         if match is not None:
#             #             print match.group(1)
#             #             print match.group(2)
#             #             print match.group(3)
#             content = None
#             if match.group(2):
#                 content = match.group(2)

#             if match.group(3):
#                 content = match.group(3)

#             if content is not None:
#                 cut_df = self.segmentor.pseg_lcut(content)
#                 for index, row in cut_df.iterrows():
#                     w = row['word']
#                     n = row['tag']
#                     print w
#                     print n
#                     if n.lower() in self.get_people_subject():
#                         self.label = self.Label.WHO
#                         self.target = w
#                         return True

#         #5
#         rule = u'(.*)' + self.get_unit_word_pre_rule()
#         #         print rule
#         pattern = re.compile(rule)
#         match = pattern.search(input_text)

#         if match is not None:
#             #             print match.group(1)

#             content = None
#             if match.group(1):
#                 content = match.group(1)

#             if content is not None:
#                 cut_df = self.segmentor.pseg_lcut(content)
#                 for index, row in cut_df.iterrows():
#                     w = row['word']
#                     n = row['tag']
#                     if n.lower() in self.get_people_subject():
#                         self.label = self.Label.WHO
#                         self.target = w
#                         return True
    
# cc = CheckWho(segmentor)
# if cc.check(u'[問卦] 科技業真的有那麼爽嗎?'):
#     print cc.label+ ' ; ' + cc.target

# 詢問一般事務

In [14]:
class CheckWhat(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckWhat, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問一般事務:
        label: what
        句型：
        1.單一個字{詞性為名詞}
        2.(什麼是|哪個是|那個是)+{詞性為名詞}
        3.{詞性為名詞}+(是什麼|是哪個|是那個)
        
        """
        self.label = ''
        self.target = ''
        
        #1
        cut_df = self.segmentor.pseg_lcut(input_text)
#         print cut_df
        if len(cut_df) == 1:
            n = cut_df['tag'][0]
#             print n
            if n.lower() in self.get_n_subject()or \
                n.lower() in self.get_eng_subject():
#                 print 'in'
                self.label = self.Label.WHAT
                self.target = cut_df['word'][0]
                return True
        
        #2
        rule = u'(什麼是|哪個是|那個是)(.*)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
#             print match.group(1)
#             print match.group(2)

            content = None
            if match.group(1):
                content = match.group(1)
            
            if match.group(2):
                content = match.group(2)

            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
#                 print 'cut_df'
#                 print cut_df
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
#                     if n.lower() not in self.get_people_subject() and \
#                             n.lower() not in self.get_location_subject():
                    if n.lower() in self.get_n_subject() or \
                        n.lower() in self.get_eng_subject():
                        self.label = self.Label.WHAT
                        self.target = w
                        return True

        #3
        rule = u'(.*)(是什麼|是哪個|是那個)'
        #         print rule
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
            content = None
            if match.group(1):
                content = match.group(1)

            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
#                     print w
#                     print n
#                     if n.lower() not in self.get_people_subject() and \
#                             n.lower() not in self.get_location_subject():
                    if n.lower() in self.get_n_subject() or \
                        n.lower() in self.get_eng_subject():
                        self.label = self.Label.WHAT
                        self.target = w
                        return True

# cc = CheckWhat(segmentor)
# if cc.check(u'發牢騷'):
#     print cc.label+ ' ; ' + cc.target

# 詢問動作

In [15]:
class CheckHow(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckHow, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問動作:
        label: how
        句型：
        1.(怎麼|怎樣|如何|教我|教導我|學|學會)+動詞
        """
        self.label = ''
        self.target = ''
        
        #1
        rule = u'(怎麼|怎樣|如何|教我|教導我|想學|學會)(.*)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
#             print match.group(1)
#             print match.group(2)

            content = match.group(2)
            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
#                     print w
#                     print n
                    if n.lower() in self.get_v_subject():
                        self.label = self.Label.HOW
                        self.target = input_text
                        return True
                    break

# cc = CheckHow(segmentor)
# if cc.check(u'給大學生的建議'):
#     print cc.label+ ' ; ' + cc.target

# 詢問數字

In [16]:
class CheckHowMany(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckHowMany, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問數字:
        label: howmany
        句型：
        1.(多少|幾)
        """
        self.label = ''
        self.target = ''
        
        #1
        rule = u'(多少|幾)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)
        #         print rule
        if match is not None:
            self.label = self.Label.HOW_MANY
            return True

# cc = CheckHowMany(u'老闆多少錢阿一個')
# if cc.check():
#     print cc.label

# 詢問狀態

In [17]:
class CheckStatus(CheckQuestion):

    def __init__(self, segmentor):
        super(CheckStatus, self).__init__()
        self.label = ''
        self.target = ''
        self.segmentor = segmentor

    def check(self, input_text):
        """
        詢問狀態:
        label: what status
        句型：
        1.多 + 形容詞
        """
        self.label = ''
        self.target = ''
        
        #1
        rule = u'(多)(.*)'
        pattern = re.compile(rule)
        match = pattern.search(input_text)

        if match is not None:
            #             print match.group(1)
            #             print match.group(2)
            content = match.group(2)
            if content is not None:
                cut_df = self.segmentor.pseg_lcut(content)
                for index, row in cut_df.iterrows():
                    w = row['word']
                    n = row['tag']
                    if n.lower() in self.get_adj_subject():
                        self.label = self.Label.STATUS
                        return True
                    break


# 詢問看法

In [18]:
class CheckFeel(CheckQuestion):
    def __init__(self, input_text):
        super(CheckFeel, self).__init__()
        self.label = ''
        self.target = ''
        self.input_text = input_text

    def check(self):  
        """
        詢問看法:
        label: feel
        句型：
        1.(感覺|看法|想法|意見|覺得)(呢|勒|類|嗎|如何|怎樣|怎麼樣)
        """
        
        self.label = ''
        self.target = '' 
        
        #1
        rule = u'(感覺|看法|想法|意見|覺得)(呢|勒|類|嗎|如何|怎樣|怎麼樣)'
        pattern = re.compile(rule)
        match = pattern.search(self.input_text)
#         print rule
        if match is not None:
            self.label = self.Label.FEEL
            return True
    
# cc = CheckFeel(u'各位酸民覺得呢')
# if cc.check():
#     print cc.label

# 其他

In [19]:
class CheckOther(CheckQuestion):

    def __init__(self):
        super(CheckOther, self).__init__()

    def check(self, input_text):
        self.label = 'other'
        self.target = ''
        return True

In [228]:
jieba_dict_path1 = "/home/charles/dataset/jieba/dict_taiwan.txt"
jieba_dict_path2 = "/home/charles/dataset/jieba/userdict.txt"
jieba_dict_path3 = "/home/charles/dataset/jieba/dict.txt.big"
jieba_dict_path4 = "/home/charles/dataset/jieba/dict.txt.small"
jieba_stopwords_path = "/home/charles/dataset/jieba/stopwords.txt"
segmentor = JiebaSegmentor(jieba_dict_path1, 
                           [jieba_dict_path2,jieba_dict_path3,jieba_dict_path4], 
                           stopwords = False, 
                           stopwords_path=jieba_stopwords_path)

Building prefix dict from /home/charles/dataset/jieba/dict_taiwan.txt ...
DEBUG:jieba:Building prefix dict from /home/charles/dataset/jieba/dict_taiwan.txt ...
Loading model from cache /tmp/jieba.u48306fa201322dcccc3d0c62898fbadc.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.u48306fa201322dcccc3d0c62898fbadc.cache
Loading model cost 0.867 seconds.
DEBUG:jieba:Loading model cost 0.867 seconds.
Prefix dict has been built succesfully.
DEBUG:jieba:Prefix dict has been built succesfully.


/home/charles/dataset/jieba/userdict.txt
/home/charles/dataset/jieba/dict.txt.big
/home/charles/dataset/jieba/dict.txt.small


In [21]:
from functools import wraps
import regex as re

In [36]:
def cache(func):
    memo = {}

    @wraps(func)
    def _wrapper(*args):
        res = memo.get(args, None)
        if res is not None:
            print 'cache exist'
            return res
        else:
            res = func(*args)
            memo[args] = res
        return res
    return _wrapper

class WikiPedia:
    
    def __init__(self, lang='zh'):
        self.__lang = lang
   
    def set_lang(self, lang):
        self.__lang = lang

    def wiki_query_list(self, query, lang):
        """
        查詢wiki單字解釋與相關詞API
        """

        url = ("https://"+lang+".wikipedia.org/w/api.php?action=opensearch&search="+query+"&utf8")    
        response = requests.post(url)
        if response.status_code == requests.codes.ok:
            result = response.json()
        #     print result[0]
            if len(result[2]) == 0:
                return None

            content = result[2][0]
            if content == u"" or u"重定向" in content:  
                print 'redirect'
                return None

            return content
        
    def wiki_query_snippet(self, query, lang):
    
        def query_api(query_str, lang):
            
            search_list = []
            url = ("https://"+lang+".wikipedia.org/w/api.php?action=query&list=search&srsearch="+
                    query_str+"&format=json&formatversion=2")    
            response = requests.post(url)
            if response.status_code == requests.codes.ok:

                result = response.json()
                if "query" in result:
                    if "search" in result["query"]:
                        search_list = result["query"]["search"]

            return search_list

        search_list = query_api(query, lang)
        if search_list:

            if search_list[0].has_key("title"):
                title = search_list[0]["title"]
                search_list = query_api(title, lang)  
                if search_list:
                    if search_list[0].has_key("snippet") and\
                        search_list[0]["snippet"]:

                        snippet = search_list[0]["snippet"]
                        snippet = snippet.split(u'。')[0]
                        snippet = re.sub(r'</?\w+[^>]*>','',snippet)+u'。'
                        return snippet
        return None
    
    @cache
    def summery(self, query):
        """
        查詢wiki API
        1.查詢wiki單字解釋與相關詞API
        2.查詢wiki單字解釋片段API
        都以中文wiki百科查詢, 當出現重定向(縮寫或只有簡體文章時發生), 改由查詢wiki單字解釋片段API
        wiki單字解釋片段API的答案不完整, 所以不考慮一開始就呼叫
        爲了提高英文單字準確性, 會使用wiki單字解釋片段API查出的title再當單字查詢一次
        """

        def check_contain_chinese(query):

            for ch in query:
                if u'\u4e00' <= ch <= u'\u9fff':
                    return True
            return False

        if not query:
            return None

        if type(query) is not unicode:
            query = query.decode('utf-8')

    #     if check_contain_chinese(query):
    #         lang = 'zh'
    #     else:
    #         lang = 'en'

        lang = 'zh'

        print 'wiki query : ' + query
        for result in [self.wiki_query_list(query, lang), 
                       self.wiki_query_snippet(query, lang)]:
            if result:
                return result
        return None

In [37]:
from collections import namedtuple
class QuestionTypeCheck:

    def __init__(self, segmentor, source='user'):
        self.segmentor = segmentor
        self.source = source
        self.check_pipeline = [CheckWhy(self.segmentor),
                              CheckWhen(self.segmentor),
                              CheckWhere(self.segmentor),
                              CheckWho(self.segmentor),
                              CheckHow(self.segmentor),
                              CheckHowMany(self.segmentor),
                              CheckStatus(self.segmentor),
                              CheckWhat(self.segmentor)]
        #                       CheckFeel(self.segmentor)]

    QuestionTypeTuple = namedtuple('QuestionTypeTuple', {
                                    'label',
                                    'target',
                                })    
    
    @property
    def source(self):
        return self.source

    @source.setter
    def source(self, source):
        self.source = source
    
    def check_question_type(self, input_text):
        """
        [ Command ] design pattern
        """ 
        if type(input_text) is not unicode:
            input_text = input_text.decode('utf-8')
        
#         print input_text
        if self.source == 'ptt':
            # ptt text clean
            input_text = input_text.replace(u'Re: ', u'')
            input_text = re.sub(u"\\[.*?]", u"", input_text)
        elif self.source == 'dcard':
            input_text = re.sub(u"\\(.*?\\)|\\{.*?}|\\[.*?]|\\（.*?）|\\【.*?】|\\#.*? |\\#.*?#", "", input_text)
              
        label = ''
        target = ''

        # 有被標注到就不往下做
        for cmd in self.check_pipeline:
            if cmd.check(input_text):
                label = cmd.label
                target = cmd.target
                break

        if label == '':
            cmd = CheckOther()
            cmd.check(input_text)
            label = cmd.label
        
        qt = self.QuestionTypeTuple(label=label,
                                   target=target)
        
#         print 'label = ' + qt.label
#         print 'target = ' + qt.target
#         if qt.label in ['what', 'where', 'who']:
#             wp = WikiPedia()
#             print wp.summery(qt.target)
#         elif qt.label in ['how']:
#             # google整句丟入
#             google_search(qt.target)
        return qt

In [38]:
def google_search(query):
    
    if not query:
        return None
    
    if type(query) is not unicode:
        query = query.decode('utf-8')
        
    print 'google query : ' + query
        
    # Google 搜尋 URL
    google_url = 'https://www.google.com.tw/search'

    # 查詢參數
    my_params = {'q': query, 'lr':'lang_zh-TW'}

    # 下載 Google 搜尋結果
    r = requests.get(google_url, params = my_params)

    # 確認是否下載成功
    if r.status_code == requests.codes.ok:
      # 以 BeautifulSoup 解析 HTML 原始碼
      soup = BeautifulSoup(r.text, 'html.parser')

      # 觀察 HTML 原始碼
      # print(soup.prettify())

      # 以 CSS 的選擇器來抓取 Google 的搜尋結果
      items = soup.select('div.g > h3.r > a[href^="/url"]')
      for i in items[:5]:
        # 標題
        print(u"標題：" + i.text)
        # 網址
        print(u"網址：" + i.get('href').lstrip('/url?q='))

In [46]:
check = QuestionTypeCheck(segmentor)
check.source = 'dcard'

In [47]:
check.check_question_type(u'Re: [新聞]黃之瀚提美台')

   tag word
0  eng   Re
1    x    :
2    x     
3   nr  黃之瀚
4    v    提
5    n   美台


QuestionTypeTuple(target='', label='other')

In [48]:
check.check_question_type(u'老闆這袋子西瓜有幾顆')

  tag word
0   n   老闆
1  zg    這
2   n   袋子
3  ns   西瓜
4   v    有
5   m   幾顆


QuestionTypeTuple(target='', label='how_many')

In [49]:
check.check_question_type(u'北京在哪')

QuestionTypeTuple(target=u'\u5317\u4eac', label='where')

In [50]:
check.check_question_type(u'蘇貞昌')

  tag word
0  nr  蘇貞昌


QuestionTypeTuple(target=u'\u8607\u8c9e\u660c', label='who')

In [51]:
check.check_question_type(u'哪個西瓜這麼大一顆')
# error......

  tag word
0   r   哪個
1  ns   西瓜
2   r   這麼
3   a    大
4   m   一顆


QuestionTypeTuple(target='', label='other')

In [52]:
check.check_question_type(u'馬英九')

  tag word
0  nr  馬英九


QuestionTypeTuple(target=u'\u99ac\u82f1\u4e5d', label='who')

In [53]:
check.check_question_type(u'關於封鎖')

  tag word
0   p   關於
1  nr   封鎖


QuestionTypeTuple(target='', label='other')

In [54]:
check.check_question_type(u'老闆這個水果多少錢呢')

  tag word
0   n   老闆
1   r   這個
2   n   水果
3  nr  多少錢
4   y    呢


QuestionTypeTuple(target='', label='how_many')

In [55]:
check.check_question_type(u'老闆這個水果多高')

  tag word
0   n   老闆
1   r   這個
2   n   水果
3   m    多
4   a    高


QuestionTypeTuple(target='', label='status')

In [56]:
check.check_question_type(u'姚明身高有多高呢')

  tag word
0  nr   姚明
1   v   身高
2   v    有
3   m    多
4   a    高
5   y    呢


QuestionTypeTuple(target='', label='status')

In [57]:
check.check_question_type(u'林百里是哪位')

  tag word
0  nr  林百里
1   v    是
2   r   哪位


QuestionTypeTuple(target=u'\u6797\u767e\u91cc', label='who')

In [245]:
check.check_question_type(u'請問apple是什麼')

QuestionTypeTuple(target=u'apple', label='what')

In [246]:
check.check_question_type(u'[問卦] 有沒有subway塑膠手套ㄉ8卦?')

QuestionTypeTuple(target='', label='other')

In [247]:
check.check_question_type(u'我想學游泳')

QuestionTypeTuple(target=u'\u6211\u60f3\u5b78\u6e38\u6cf3', label='how')

In [248]:
check.check_question_type(u'連勝文')

QuestionTypeTuple(target=u'\u9023\u52dd\u6587', label='who')

In [249]:
check.check_question_type('給大學生的建議')

QuestionTypeTuple(target='', label='other')

In [250]:
# ppp

# question type setting

In [251]:
dao = CassandraDAO('BACKUP')
HELPER_KEYSPACE = 'nlp_keyspace'
DCARD_ARTICLE_TABLE = 'dcard_article'
DCARD_ARTICLE_TEST_TABLE = 'dcard_article_test'

contact_points = 192.168.0.121,192.168.0.122,192.168.0.52


In [ ]:
cql = 'select * from ' + DCARD_ARTICLE_TEST_TABLE + ';'
## select range
# cql = ("select * from nlp_keyspace.dcard_article_test " +
#         "where article_id > 225705809 and article_id < 225800010 allow filtering;")
pd_df = dao.execCQLSelectToPandasDF(HELPER_KEYSPACE, cql)
pd_df

In [275]:
def set_question_type(row):
    title = row['question']
    qt = check.check_question_type(title)
    return qt.label

pd_df['question_type'] = pd_df.apply(set_question_type, axis=1)
pd_df

/home/charles/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,article_id,board,category,content,crawl_date,create_date,question,question_target,question_type,title
0,224297810,job,None,想請問在健身房的工作內容有什麼呢需要做什麼 還是要會什麼技能櫃檯/教練,2019-02-15 07:49:24.035,2016-07-03T16:51:05.200Z,健身房工作經驗,None,other,健身房工作經驗（問）
1,225961291,talk,None,首次發文，手機排版，請見諒（跟朋友借的帳號）首先，刺青是一門藝術並不是每個刺青的人都是屁孩 ...,2019-02-19 08:25:10.219,2017-03-11T09:36:15.788Z,刺青的意義,None,other,#圖 刺青的意義
2,225196435,food,None,,2019-02-13 01:39:36.743,2016-11-15T03:16:45.628Z,蜂蜜真假,None,other,蜂蜜真假
3,224746504,talk,None,各種淡紫控啊~覺得淡紫超美的～～😘還在繼續搜集中～(燈光比較不好，淡紫色沒有很明顯…😭)－愛...,2019-02-19 06:09:49.412,2016-09-12T14:33:05.434Z,愛各種淡紫色,None,other,愛各種淡紫色
4,230071785,trending,None,剛剛看到謝和炫出的這首歌，聽完簡直感動的說不出話來....台北年輕人這次一定要出來投票衝高投...,2019-02-15 02:46:30.961,2018-11-15T19:22:07.846Z,台灣的未來交給柯文哲！,None,other,台灣的未來交給柯文哲！
5,226029225,food,None,不知道有沒有人記得前幾天的單身狗廚房肉篇XD這次小廚跟朋友一行人去南投的某茶園露營(不透漏確...,2019-02-13 03:28:19.805,2017-03-22T04:04:36.302Z,單身狗廚房x野外露營篇,None,other,#圖 單身狗廚房x野外露營篇
6,226491861,food,None,想跟大家分享這間超好吃的日本料理店！他位於現在很多人要去的小琉球的乘船處----東港！我個人...,2019-02-13 03:54:05.395,2017-05-28T16:40:26.874Z,龍允日式手作壽司,None,other,#食記 #東港 龍允日式手作壽司
7,229830476,language,None,請問第三句 那個「となり 」是翻成登山「時」的意思嗎？,2019-02-15 07:05:51.092,2018-10-11T12:06:50.282Z,日文翻譯問題,None,other,日文翻譯問題
8,465696,talk,None,ㄜ 因為突然想到很想知道，雖然感覺這樣問有點奇怪= =前幾個星期有看到新聞在介紹，有個女生之...,2019-02-18 06:33:17.498,2015-08-17T13:33:21.449Z,尋~一個卡片遊戲 給背包客專用 (電視上看到,None,other,尋~一個卡片遊戲(?) 給背包客專用 (電視上看到
9,230081344,trending,None,本人台中人從小有嚴重過敏體質，鼻子容易塞住，也很容易流鼻水，更容易狂打噴嚏，成年之後狀況已經...,2019-02-15 02:47:31.216,2018-11-17T11:02:23.407Z,我不想吸髒空氣,None,other,我不想吸髒空氣


In [276]:
def set_question_target(row):
    qt = check.check_question_type(row['question'])
    return qt.target

pd_df['question_target'] = pd_df.apply(set_question_target, axis=1)
pd_df

/home/charles/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,article_id,board,category,content,crawl_date,create_date,question,question_target,question_type,title
0,224297810,job,None,想請問在健身房的工作內容有什麼呢需要做什麼 還是要會什麼技能櫃檯/教練,2019-02-15 07:49:24.035,2016-07-03T16:51:05.200Z,健身房工作經驗,,other,健身房工作經驗（問）
1,225961291,talk,None,首次發文，手機排版，請見諒（跟朋友借的帳號）首先，刺青是一門藝術並不是每個刺青的人都是屁孩 ...,2019-02-19 08:25:10.219,2017-03-11T09:36:15.788Z,刺青的意義,,other,#圖 刺青的意義
2,225196435,food,None,,2019-02-13 01:39:36.743,2016-11-15T03:16:45.628Z,蜂蜜真假,,other,蜂蜜真假
3,224746504,talk,None,各種淡紫控啊~覺得淡紫超美的～～😘還在繼續搜集中～(燈光比較不好，淡紫色沒有很明顯…😭)－愛...,2019-02-19 06:09:49.412,2016-09-12T14:33:05.434Z,愛各種淡紫色,,other,愛各種淡紫色
4,230071785,trending,None,剛剛看到謝和炫出的這首歌，聽完簡直感動的說不出話來....台北年輕人這次一定要出來投票衝高投...,2019-02-15 02:46:30.961,2018-11-15T19:22:07.846Z,台灣的未來交給柯文哲！,,other,台灣的未來交給柯文哲！
5,226029225,food,None,不知道有沒有人記得前幾天的單身狗廚房肉篇XD這次小廚跟朋友一行人去南投的某茶園露營(不透漏確...,2019-02-13 03:28:19.805,2017-03-22T04:04:36.302Z,單身狗廚房x野外露營篇,,other,#圖 單身狗廚房x野外露營篇
6,226491861,food,None,想跟大家分享這間超好吃的日本料理店！他位於現在很多人要去的小琉球的乘船處----東港！我個人...,2019-02-13 03:54:05.395,2017-05-28T16:40:26.874Z,龍允日式手作壽司,,other,#食記 #東港 龍允日式手作壽司
7,229830476,language,None,請問第三句 那個「となり 」是翻成登山「時」的意思嗎？,2019-02-15 07:05:51.092,2018-10-11T12:06:50.282Z,日文翻譯問題,,other,日文翻譯問題
8,465696,talk,None,ㄜ 因為突然想到很想知道，雖然感覺這樣問有點奇怪= =前幾個星期有看到新聞在介紹，有個女生之...,2019-02-18 06:33:17.498,2015-08-17T13:33:21.449Z,尋~一個卡片遊戲 給背包客專用 (電視上看到,,other,尋~一個卡片遊戲(?) 給背包客專用 (電視上看到
9,230081344,trending,None,本人台中人從小有嚴重過敏體質，鼻子容易塞住，也很容易流鼻水，更容易狂打噴嚏，成年之後狀況已經...,2019-02-15 02:47:31.216,2018-11-17T11:02:23.407Z,我不想吸髒空氣,,other,我不想吸髒空氣


In [280]:
filter_pd = pd_df[pd_df['question_type'] != 'other']
print len(filter_pd)
# filter_pd[['article_id','board','title','question','question_type','question_target']]
for i,r in filter_pd.iterrows():
    print r['title'] + ' | ' + r['question'] + ' | ' + r['question_type'] + ' | ' + r['question_target']

309
＃打工  寒假如何找超過基本工資的打工？ | ＃打工  寒假如何找超過基本工資的打工？ | how | ＃打工  寒假如何找超過基本工資的打工？
未來的出路在哪裏 | 未來的出路在哪裏 | where | 
#評論 國民黨產帳怎麼算？退將：民進黨也曾被餵黨產奶水 | 國民黨產帳怎麼算？退將：民進黨也曾被餵黨產奶水 | how | 國民黨產帳怎麼算？退將：民進黨也曾被餵黨產奶水
發發牢騷 | 發發牢騷 | what | 發發牢騷
公投怎麼投？ | 公投怎麼投？ | how | 公投怎麼投？
畢業旅行～有哪些地方可以選擇呢 | 畢業旅行～有哪些地方可以選擇呢 | where | 
錢該怎麼算 | 錢該怎麼算 | how | 錢該怎麼算
智力測驗 | 智力測驗 | what | 智力測驗
拉K的味道怎麼辨別 | 拉K的味道怎麼辨別 | how | 拉K的味道怎麼辨別
（問）整棟只有一個電表 那電費大家會如何算呢 | 整棟只有一個電表 那電費大家會如何算呢 | how | 整棟只有一個電表 那電費大家會如何算呢
怎麼判別甲甲 | 怎麼判別甲甲 | how | 怎麼判別甲甲
台北下著雨，餓著肚子的人何處去? | 台北下著雨，餓著肚子的人何處去? | where | 台北
智利櫻桃現在市價大約多少呢？ | 智利櫻桃現在市價大約多少呢？ | how_many | 
為什麼未滿18歲不能看限制級電影 | 為什麼未滿18歲不能看限制級電影 | why | 
大家畢業多久才找到工作? | 大家畢業多久才找到工作? | status | 
劍靈 | 劍靈 | where | 劍靈
【提問】最近CPBL熱身賽的主播跟球評到底是誰....? | 最近CPBL熱身賽的主播跟球評到底是誰....? | who | 
這個甜點哪裡買的到呢 | 這個甜點哪裡買的到呢 | where | 
驅逐出場 | 驅逐出場 | what | 驅逐出場
真的窮到不知如何是好，求分享 | 真的窮到不知如何是好，求分享 | how | 真的窮到不知如何是好，求分享
大學生必看！「甜課」的日文怎麼說呢？ | 大學生必看！「甜課」的日文怎麼說呢？ | how | 大學生必看！「甜課」的日文怎麼說呢？
要怎麼加入民進黨黨工啊 | 要怎麼加入民進黨黨工啊 | how | 要怎麼加入民進黨黨工啊
#問#西班牙文 | 西

月亮 | 月亮 | what | 月亮
#徵才 工讀生 | 工讀生 | what | 工讀生
簽名檔 | 簽名檔 | what | 簽名檔
統神到底是誰? | 統神到底是誰? | who | 
車禍問題 該怎麼處理 | 車禍問題 該怎麼處理 | how | 車禍問題 該怎麼處理
多少人把這邊當相親網站 ? | 多少人把這邊當相親網站 ? | how_many | 
#更 到底值多少錢？ | 到底值多少錢？ | how_many | 
#問 股票 | 股票 | what | 股票
大家房租都多少錢啊？? | 大家房租都多少錢啊？? | how_many | 
雲林虎尾有哪裡剪髮不錯的嗎? | 雲林虎尾有哪裡剪髮不錯的嗎? | where | 雲林
災後 | 災後 | what | 災後
遠傳入主中嘉為什麼不好? | 遠傳入主中嘉為什麼不好? | why | 
#圖#花蓮#台東 美食 | 美食 | what | 美食
選情 | 選情 | what | 選情
#問 如何阻止別人辦信用卡 | 如何阻止別人辦信用卡 | how | 如何阻止別人辦信用卡
【轉貼】低薪?沒人才? 台灣哪裡沒人才了 人材多的是 | 低薪?沒人才? 台灣哪裡沒人才了 人材多的是 | where | 
誰是統獨教父？請理性看待統獨 | 誰是統獨教父？請理性看待統獨 | who | 
為什麼大家跑操場都跑逆時鐘 | 為什麼大家跑操場都跑逆時鐘 | why | 
趕潮流之如何分辨台科生 | 趕潮流之如何分辨台科生 | how | 趕潮流之如何分辨台科生
學了第二外語，才知道什麼是真正的學語言 | 學了第二外語，才知道什麼是真正的學語言 | what | 學
沖繩 | 沖繩 | what | 沖繩
洗手檯 | 洗手檯 | what | 洗手檯
請問國外adidas nmd R1 哪裡買？ | 請問國外adidas nmd R1 哪裡買？ | where | 
請問新手怎麼挑選股票 | 請問新手怎麼挑選股票 | how | 請問新手怎麼挑選股票
為什麼做甜點? | 為什麼做甜點? | why | 
Snapchat | Snapchat | what | Snapchat
香腸為什麼叫煙強？ | 香腸為什麼叫煙強？ | why | 
出國考察 | 出國考察 | what | 出國考察
Follow | Fol

In [278]:
ppp

NameError: name 'ppp' is not defined

In [281]:
#insert 
DCARD_ARTICLE_TABLE = 'dcard_article_test'
# cql = cql + 'BEGIN BATCH '
for index, row in pd_df.iterrows():

    cql = ("insert into " + DCARD_ARTICLE_TABLE + "(article_id, question, question_type) " + 
          "values(" + str(row['article_id']) + ",'" + 
          row['question'] + "','" + row['question_type'] + "');")
#     print cql
#     dao.execCQL(HELPER_KEYSPACE, cql)
    
# cql = cql + ' APPLY BATCH '
# print cql

insert into dcard_article_test(article_id, question, question_type) values(224297810,'健身房工作經驗','other');
insert into dcard_article_test(article_id, question, question_type) values(225961291,'刺青的意義','other');
insert into dcard_article_test(article_id, question, question_type) values(225196435,'蜂蜜真假','other');
insert into dcard_article_test(article_id, question, question_type) values(224746504,'愛各種淡紫色','other');
insert into dcard_article_test(article_id, question, question_type) values(230071785,'台灣的未來交給柯文哲！','other');
insert into dcard_article_test(article_id, question, question_type) values(226029225,'單身狗廚房x野外露營篇','other');
insert into dcard_article_test(article_id, question, question_type) values(226491861,'龍允日式手作壽司','other');
insert into dcard_article_test(article_id, question, question_type) values(229830476,'日文翻譯問題','other');
insert into dcard_article_test(article_id, question, question_type) values(465696,'尋~一個卡片遊戲 給背包客專用 (電視上看到','other');
insert into dcard_article_test(article_id

insert into dcard_article_test(article_id, question, question_type) values(658066,'畢業旅行～有哪些地方可以選擇呢','where');
insert into dcard_article_test(article_id, question, question_type) values(225084801,'關於公關這個工作 2','other');
insert into dcard_article_test(article_id, question, question_type) values(224721291,'日式彩虹胡麻烏龍麵','other');
insert into dcard_article_test(article_id, question, question_type) values(224911186,'涉性侵少女 林依晨前男友遭訴','other');
insert into dcard_article_test(article_id, question, question_type) values(1173439,'嘉義微食記分享 part2','other');
insert into dcard_article_test(article_id, question, question_type) values(228808840,'請問例句翻譯','other');
insert into dcard_article_test(article_id, question, question_type) values(224076541,'誠徵 工讀生','other');
insert into dcard_article_test(article_id, question, question_type) values(207263,'有推薦的畢業禮物嘛','other');
insert into dcard_article_test(article_id, question, question_type) values(227789040,'兩日遊吃了什麼😋','other');
insert into dcard_article_test(artic

insert into dcard_article_test(article_id, question, question_type) values(224658723,'有沒有躺著玩手機結果掉下來打到自己臉的掛','other');
insert into dcard_article_test(article_id, question, question_type) values(228253086,'別相信任何人','other');
insert into dcard_article_test(article_id, question, question_type) values(216768478,'#圖🍲🍲鮮蔬雞肉燴飯🍲🍲','other');
insert into dcard_article_test(article_id, question, question_type) values(229913174,'幫民進黨思考兩岸新套路','other');
insert into dcard_article_test(article_id, question, question_type) values(216760,'Re: 「請孫文、黨旗、黨歌退出校園」公民行動連署','other');
insert into dcard_article_test(article_id, question, question_type) values(1000643,'努力減肥想考空服','other');
insert into dcard_article_test(article_id, question, question_type) values(227498538,'謊言軍隊（文長','other');
insert into dcard_article_test(article_id, question, question_type) values(230194224,'華航飛機餐','other');
insert into dcard_article_test(article_id, question, question_type) values(225210291,'求指點迷津文長','other');
insert into dcard_arti

insert into dcard_article_test(article_id, question, question_type) values(988319,'可以不要挺大麻嗎?','other');
insert into dcard_article_test(article_id, question, question_type) values(224631271,'速可達１２５比較','other');
insert into dcard_article_test(article_id, question, question_type) values(1114017,'聯合國將禁止日本性暴力ACG','other');
insert into dcard_article_test(article_id, question, question_type) values(224787551,'同理心？','other');
insert into dcard_article_test(article_id, question, question_type) values(224464674,'蝦仁炒飯！！！GAN','other');
insert into dcard_article_test(article_id, question, question_type) values(228346629,'  金門古法道地蚵仔煎','other');
insert into dcard_article_test(article_id, question, question_type) values(40475,'深夜串詞－曖昧','other');
insert into dcard_article_test(article_id, question, question_type) values(226826924,'7/18 乳摸','other');
insert into dcard_article_test(article_id, question, question_type) values(225777148,'半年特休','other');
insert into dcard_article_test(article_id, question, 

insert into dcard_article_test(article_id, question, question_type) values(224374928,'妳最受不了妳男朋友做什麼？','other');
insert into dcard_article_test(article_id, question, question_type) values(224524957,'人手拿哀鳳不過就是小確辛','other');
insert into dcard_article_test(article_id, question, question_type) values(1056577,'討厭師大檢察官的到底是抱持著什麼樣的想法??','other');
insert into dcard_article_test(article_id, question, question_type) values(227556810,'廣告配唱、DEMO歌手接案','other');
insert into dcard_article_test(article_id, question, question_type) values(948706,'關於阿嬤的藥','other');
insert into dcard_article_test(article_id, question, question_type) values(225696831,'擦肩而過','other');
insert into dcard_article_test(article_id, question, question_type) values(47727586,'Re 不是歧視肥宅，只是純粹不喜歡動漫','other');
insert into dcard_article_test(article_id, question, question_type) values(181809,'自己做超豐盛晚餐','other');
insert into dcard_article_test(article_id, question, question_type) values(124018699,'轉  爽翻了！台灣詐欺犯抵中國　全自由了','other');
insert int

...

insert into dcard_article_test(article_id, question, question_type) values(224767257,'系邊是....？','other');
insert into dcard_article_test(article_id, question, question_type) values(228816447,'五月第一天班想蹺班然後離職會扣錢嗎？','other');
insert into dcard_article_test(article_id, question, question_type) values(150557768,'鄭捷的死刑，廢死聯盟也有責任！','other');
insert into dcard_article_test(article_id, question, question_type) values(224300561,'一個月的甜點時光','other');
insert into dcard_article_test(article_id, question, question_type) values(510658,'團康競賽後的懲罰方式？','other');
insert into dcard_article_test(article_id, question, question_type) values(468337,'我幫妳領養了一個小孩','other');
insert into dcard_article_test(article_id, question, question_type) values(320454,'期中考V.S期末考','other');
insert into dcard_article_test(article_id, question, question_type) values(225731979,'關於你一生中的工作選擇','other');
insert into dcard_article_test(article_id, question, question_type) values(230145076,'臺灣人啊，請想一想......','other');
insert into dcard_arti

insert into dcard_article_test(article_id, question, question_type) values(172314972,'韓之棧韓式傳統料理～～so spicy','other');
insert into dcard_article_test(article_id, question, question_type) values(190317134,'母親節送到送歪歪!!!','other');
insert into dcard_article_test(article_id, question, question_type) values(51542884,'難道是命中注定？','other');
insert into dcard_article_test(article_id, question, question_type) values(108895534,'離譜大學研究員　教14歲女兒同學課業教上床','other');
insert into dcard_article_test(article_id, question, question_type) values(228295642,'工讀實習有年終嗎？','other');
insert into dcard_article_test(article_id, question, question_type) values(226030590,'關於送禮','other');
insert into dcard_article_test(article_id, question, question_type) values(27771484,'怎麼寫阿阿阿','how');
insert into dcard_article_test(article_id, question, question_type) values(224044102,'我的手機應用程式','other');
insert into dcard_article_test(article_id, question, question_type) values(227964480,'聖誕禮物交換','other');
insert into dcard_article_test

insert into dcard_article_test(article_id, question, question_type) values(228920198,'德明財經外的雞蛋糕','other');
insert into dcard_article_test(article_id, question, question_type) values(83200424,' 抹茶紅豆蛋糕捲','other');
insert into dcard_article_test(article_id, question, question_type) values(99739803,'宿舍百態','other');
insert into dcard_article_test(article_id, question, question_type) values(224526667,'「立委不知民間疾苦」教官若退出校園 私校師很焦慮','other');
insert into dcard_article_test(article_id, question, question_type) values(227080253,'請問如何找打工？','how');
insert into dcard_article_test(article_id, question, question_type) values(230610586,'非核家園終究是白日作夢','other');
insert into dcard_article_test(article_id, question, question_type) values(53056,'關於Spotify 有幾個小問題','how_many');
insert into dcard_article_test(article_id, question, question_type) values(228576584,'習近平同金正恩舉行會談','other');
insert into dcard_article_test(article_id, question, question_type) values(229660529,'打工面試','other');
insert into dcard_article_te

insert into dcard_article_test(article_id, question, question_type) values(227845816,'電子鍋還是快鍋','other');
insert into dcard_article_test(article_id, question, question_type) values(228978851,'長榮地勤 #營業人員','other');
insert into dcard_article_test(article_id, question, question_type) values(229104400,'問 iCash 2.0','other');
insert into dcard_article_test(article_id, question, question_type) values(224461000,'爸爸的背❤','other');
insert into dcard_article_test(article_id, question, question_type) values(224530575,'江春男因酒駕請辭　蔡英文：人民對執政團隊要求很高','other');
insert into dcard_article_test(article_id, question, question_type) values(230676015,'要看情況才能發揮的同理心？','other');
insert into dcard_article_test(article_id, question, question_type) values(1032900,'吵死了~~~~','other');
insert into dcard_article_test(article_id, question, question_type) values(228201200,'求解😭Ampule碎在藥裡','other');
insert into dcard_article_test(article_id, question, question_type) values(224384638,'圖*五步驟簡易食譜*單人獨享餐🙆','other');
insert into dc

insert into dcard_article_test(article_id, question, question_type) values(230500492,'藏壽司','other');
insert into dcard_article_test(article_id, question, question_type) values(226547192,'請教英文翻譯問題','other');
insert into dcard_article_test(article_id, question, question_type) values(228952699,'蜜橙磅蛋糕','other');
insert into dcard_article_test(article_id, question, question_type) values(228673187,'剛到職，卻想離職！','other');
insert into dcard_article_test(article_id, question, question_type) values(229950650,'其邁哥的政見 一通電話醫生護士到家服務','other');
insert into dcard_article_test(article_id, question, question_type) values(226695405,'問 工時問題','other');
insert into dcard_article_test(article_id, question, question_type) values(761886,'只關心巴黎錯了嗎？　－　請別苛責只關心巴黎的你我','other');
insert into dcard_article_test(article_id, question, question_type) values(228814961,'BAKE CODE 烘焙密碼西湖店','other');
insert into dcard_article_test(article_id, question, question_type) values(706757,'Re 陸生健保','other');
insert into dcard_article_

In [ ]:
cql = 'select * from dcard_article_test;'
pd_df = dao.execCQLSelectToPandasDF(HELPER_KEYSPACE, cql)
pd_df